In [57]:
import numpy as np

# Define the arrays for treasure and hunters
treasure = np.array([
            24, 70, 41, 21, 60, 
            47, 82, 87, 80, 35, 
            73, 89, 100, 90, 17, 
            77, 83, 85, 79, 55,
            12, 27, 52, 15, 30])

hunters = np.array([
            2, 4, 3, 2, 4, 
            3, 5, 5, 5, 3, 
            4, 5, 8, 7, 2, 
            5, 5, 5, 5, 4,
            2, 3, 4, 2, 3])

#treasure = np.array([24,70,47,82])
#hunters = np.array([2,4,3,5])

# Calculate the number of elements in the hunters array
n = len(hunters)

# Initialize the array y with zeros
y = np.zeros(n)

# Assign 100 to the last element of y
y[-1] = 100

# Loop to calculate differences as per the given formula
for i in range(n - 1):
    y[i] = hunters[i+1] / (treasure[i+1] * 7500) - hunters[i] / (treasure[i] * 7500)

# Print the resulting array y
#print(y)

A = np.zeros((n, n))

# Loop to populate the matrix for the diagonal and off-diagonal elements
for i in range(n - 1):
    A[i][i] = 1 / (treasure[i] * 7500)
    A[i][i+1] = -1 / (treasure[i+1] * 7500)

# Set the last row to all ones
A[n-1] = np.ones(n)

# Print the resulting matrix A
#print(A)

In [58]:
import numpy as np
from scipy.optimize import linprog

# Linear programming does not directly support equality constraints.
# Thus, we must separate A into two halves to express them as two inequality sets.
# We will minimize a zero function (as we're only interested in finding a feasible x).
c = np.zeros(n)  # Coefficients of the dummy objective function

# Bounds for each x_i: 0 < x_i < 1
x_bounds = [(0, 100) for _ in range(n)]

# Define constraints (Ax = y converted into Ax <= y and Ax >= y)
res = linprog(c, A_eq=A, b_eq=y, bounds=x_bounds, method='highs')

if res.success:
    print("Solution x:", res.x)
else:
    print("No solution found:", res.message)


No solution found: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)


In [59]:
import numpy as np

# Try solving using least squares to see if a solution is fundamentally possible
x_ls, residuals, rank, s = np.linalg.lstsq(A, y, rcond=None)

# Check if the solution x_ls lies within the desired bounds
solution_within_bounds = np.all((x_ls > 0) & (x_ls < 100))

print("Least Squares Solution:", x_ls)
print("Solution within bounds:", solution_within_bounds)


Least Squares Solution: [ 1.35429769  5.78336827  2.73025856  0.93501048  4.38574423  3.56883298
  6.46051712  7.15932914  6.18099231  1.89168414  6.20265549  7.43885395
  5.97624039  5.57861635  0.37596087  5.7617051   6.60027952  6.87980433
  6.04122991  3.68693222 -0.32285115  0.77358491  3.267645    0.09643606
  1.19287212]
Solution within bounds: False


In [60]:
print(x_ls.reshape((5, 5)))

[[ 1.35429769  5.78336827  2.73025856  0.93501048  4.38574423]
 [ 3.56883298  6.46051712  7.15932914  6.18099231  1.89168414]
 [ 6.20265549  7.43885395  5.97624039  5.57861635  0.37596087]
 [ 5.7617051   6.60027952  6.87980433  6.04122991  3.68693222]
 [-0.32285115  0.77358491  3.267645    0.09643606  1.19287212]]


In [65]:
'''
Let us now mix this optimal distribution with other distributions
By the study from Jannik:
Naive: 19%
Optimists:3%
Spoilers: 1%
Rational: 62%
Mean (irrational): 15%

We will adapt these probabilities/playing styles to:
Spoilers: 10%
Naive: 25%
Optimists: 30%
Rational: 30%
Meta-Rational: 5%
'''

'\nLet us now mix this optimal distribution with other distributions\nBy the study from Jannik:\nNaive: 19%\nOptimists:3%\nSpoilers: 1%\nRational: 62%\nMean (irrational): 15%\n\nWe will adapt these probabilities/playing styles to:\nSpoilers: 5%\nNaive: 25%\nOptimists: 5%\nRational:55%\nMeta-Rational:10%\n'

In [88]:
'''
Spoilers will just put all their guesses on the highest three tiles: 100, 90, and 89
'''
spoilers = 100*np.array([
            0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 
            0, 1/3, 1/3, 1/3, 0, 
            0, 0, 0, 0, 0,
            0, 0, 0, 0, 0])

In [89]:
'''
Naive Players will uniformly pick between all numbers
'''

naive = 4*np.ones(n)

In [84]:
'''
Optimists will just assume everyone else plays naive,
and pick the tiles with highest payoff under these circumstances.
'''
payout = 7500*treasure/(hunters+4)
highest_indices = np.argsort(payout)[-5:]
print(highest_indices)

[10 16 17  7 11]


In [90]:
'''
This leads us to define:
'''
optimists = 20*np.array([
            0, 0, 0, 0, 0, 
            0, 0, 1, 0, 0, 
            1, 1, 0, 0, 0, 
            0, 1, 1, 0, 0,
            0, 0, 0, 0, 0])


In [91]:
'''
Rational players will assume everyone else is rational and hence they will follow the probabilities outlined:
'''
rational = x_ls

In [111]:
'''
Meta Rational Players will do a similar calculation to us and find the optimum under this calculation.
'''
distr = (1/0.95)*(0.10*spoilers + 0.25*naive + 0.30*optimists + 0.3*rational)
meta_payout = payout = 7500*treasure/(hunters+distr)
print(np.argsort(payout)[-5:])

[15 18  8  6  1]


In [112]:
meta_rational = 20*np.array([
            0, 1, 0, 0, 0, 
            0, 1, 0, 1, 0, 
            0, 0, 0, 0, 0, 
            1, 0, 0, 1, 0,
            0, 0, 0, 0, 0])

In [113]:
#Now calculate true payout
true_distr = 0.95*distr + 0.05*meta_rational
true_payout = 7500*treasure/(hunters+true_distr)
print(true_payout.round(1).reshape(5,5))

[[52843.4 67873.2 63808.9 48010.9 71250.7]
 [69517.7 68806.1 46120.2 67763.7 57471.2]
 [42571.2 38001.7 53092.8 51895.5 40960.1]
 [66162.5 44527.6 45328.7 67235.  67555.6]
 [31000.9 47848.9 65214.2 37141.8 51630.8]]


In [114]:
print(true_distr.round(1).reshape(5,5))

[[ 1.4  3.7  1.8  1.3  2.3]
 [ 2.1  3.9  9.1  3.9  1.6]
 [ 8.9 12.6  6.1  6.   1.1]
 [ 3.7  9.   9.1  3.8  2.1]
 [ 0.9  1.2  2.   1.   1.4]]
